In [1]:
import keras

from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import sgd
import datetime
import tensorflow as tf
from keras import backend as k
from keras.callbacks import ModelCheckpoint,CSVLogger,ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 32
num_classes = 10
epochs = 50
l = 18
num_filter = 24
compression = 0.5
dropout_rate = 0.2

In [4]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 28s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:

input = Input(shape=(img_height, img_width, channel,))
Conv2D_7x1 = Conv2D(num_filter,(7,1),use_bias=False,padding='same')(input)
Conv2D_1x7 = Conv2D(num_filter,(1,7),use_bias=False,padding='same')(Conv2D_7x1)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(Conv2D_1x7)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [0]:
filepath="A4weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=5, min_lr=0.001)
trainlog = CSVLogger("training.log")
callbacks_list = [checkpoint,trainlog,reduce_lr]

In [13]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   504         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 24)   4032        conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 32, 32, 24)   5184        conv2d_2[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=sgd(momentum=0.9),
              metrics=['accuracy'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.25,
    horizontal_flip=True)

validation_data_gen = ImageDataGenerator()
train_generator = train_data_gen.flow(x_train, y_train, batch_size=batch_size)
validation_generator = validation_data_gen.flow(x_test, y_test, batch_size=batch_size)

In [18]:

model.fit_generator(train_generator, 
                    steps_per_epoch=train_generator.n//batch_size, 
                    epochs=epochs, 
                    callbacks=callbacks_list, 
                    verbose =1,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.n//batch_size)

Epoch 1/50
1562/1562 [==============================] - 466s 298ms/step - loss: 0.5589 - acc: 0.8062 - val_loss: 0.6009 - val_acc: 0.8304

Epoch 00001: val_acc did not improve from 0.84425
Epoch 2/50
1562/1562 [==============================] - 464s 297ms/step - loss: 0.5594 - acc: 0.8059 - val_loss: 0.5224 - val_acc: 0.8463

Epoch 00002: val_acc improved from 0.84425 to 0.84625, saving model to A4weights.best.hdf5
Epoch 3/50
1562/1562 [==============================] - 466s 298ms/step - loss: 0.5552 - acc: 0.8074 - val_loss: 0.6045 - val_acc: 0.8299

Epoch 00003: val_acc did not improve from 0.84625
Epoch 4/50
1562/1562 [==============================] - 467s 299ms/step - loss: 0.5450 - acc: 0.8125 - val_loss: 0.6093 - val_acc: 0.8252

Epoch 00004: val_acc did not improve from 0.84625
Epoch 5/50
1562/1562 [==============================] - 468s 299ms/step - loss: 0.5403 - acc: 0.8133 - val_loss: 0.5930 - val_acc: 0.8331

Epoch 00005: val_acc did not improve from 0.84625
Epoch 6/50
156

In [0]:
from keras.models import load_model
model = load_model("A4weights.best.hdf5")

In [0]:
model.fit_generator(train_generator, 
                    steps_per_epoch=train_generator.n//batch_size, 
                    epochs=epochs, 
                    callbacks=callbacks_list, 
                    verbose =1,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.n//batch_size)

In [0]:
from keras.models import load_model
model = load_model("A4weights.best.hdf5")

In [0]:
model.fit_generator(train_generator, 
                    steps_per_epoch=train_generator.n//batch_size, 
                    epochs=epochs, 
                    callbacks=callbacks_list, 
                    verbose =1,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.n//batch_size)